In [ ]:
%%capture
!pip install jsonlines
!pip install evaluate
!pip install bert_score

In [ ]:
import pickle

# To load back the saved data
with open('predictions_references.pkl', 'rb') as file:
    data = pickle.load(file)
    fined_predictions = data['fined_predictions']
    fined_references = data['fined_references']
    predictions = data['predictions']
    references = data['references']

In [ ]:
def custom_metric(predictions, references):
    valid = 0
    correct = 0

    for prediction, reference in zip(predictions, references):
        try:
            reference = eval(reference)
            keys = set(reference.keys()) - set(["summary", "location", "specialization", "industry", "leadership_experience", "project", "team_experience", "other"])
            total_valid_fail = len(references)
            total_correct_wrong = len(keys) * total_valid_fail

            prediction = eval(prediction)

            valid += 1
            for key in keys:
                if reference[key] == prediction[key]:
                    correct += 1
                else:
                    pass
        except:
            pass
    return float(f"{valid/total_valid_fail * 100:.2f}"), float(f"{correct/total_correct_wrong * 100:.2f}")

In [ ]:
import evaluate
import numpy as np

# Load metrics
bertscore_metric = evaluate.load("bertscore")

def compute_metrics(predictions, references):
    bertscore = bertscore_metric.compute(predictions=predictions, references=references, lang="en")
    bertscore = {key: np.mean(bertscore[key]) for key in bertscore if key != "hashcode"}
    valid, correct = custom_metric(predictions=predictions, references=references)

    return {
        "bertscore": bertscore,
        "valid": valid,
        "correct": correct,
        }

In [ ]:
fined_results = compute_metrics(predictions=fined_predictions, references=fined_references)
results = compute_metrics(predictions=predictions, references=references)

In [ ]:
from pprint import pprint
print("fined_results:\n")
pprint(fined_results)
print("\n\nresults:\n")
pprint(results)

In [ ]:
import json

# Save to file
with open('fined_results.json', 'w') as file:
    json.dump(fined_results, file)

with open('results.json', 'w') as file:
    json.dump(results, file)

In [ ]:
from google.colab import files

file_path = '/content/fined_results.json'

# Trigger download
files.download(file_path)

file_path = '/content/results.json'

# Trigger download
files.download(file_path)